In [5]:
import numpy as np
import pandas as pd

In [6]:
# 0
data = pd.read_csv("RAW_Austin-Texas_AnimalCenter-Intakes.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128767 entries, 0 to 128766
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         128767 non-null  object
 1   Name              88687 non-null   object
 2   DateTime          128767 non-null  object
 3   MonthYear         128767 non-null  object
 4   Found Location    128767 non-null  object
 5   Intake Type       128767 non-null  object
 6   Intake Condition  128767 non-null  object
 7   Animal Type       128767 non-null  object
 8   Sex upon Intake   128766 non-null  object
 9   Age upon Intake   128767 non-null  object
 10  Breed             128767 non-null  object
 11  Color             128767 non-null  object
dtypes: object(12)
memory usage: 11.8+ MB


<b>REMOVE duplicate rows, FILL null values</b>

In [7]:
# 
data.drop_duplicates(inplace=True)

In [8]:
# 
data.drop("MonthYear", axis=1, inplace=True)

In [9]:
# 
data["Name"].fillna("Unknown", inplace=True)
data["Name"] = data["Name"].str.replace("*", "", regex=False)
data.loc[data["Name"]=="", "Name"] = "Unknown"

In [10]:
# 
data["Sex upon Intake"].fillna("Unknown", inplace=True)

<b>ADJUST & CREATE additional COLUMNS</b>

In [11]:
#
data.loc[data["Name"]==data["Animal ID"], "Name"] = "Unknown"

In [12]:
# 
data[["in_date", "in_time1", "in_time2"]] = data["DateTime"].str.split(pat=" ", n=2, expand=True)

In [13]:
# 
data["in_date"] = pd.to_datetime(data["in_date"], format="%m/%d/%Y").dt.strftime("%d/%m/%Y")
data["in_date"] = pd.to_datetime(data["in_date"], format="%d/%m/%Y")
data.drop("DateTime", axis=1, inplace=True)

In [14]:
#
data.rename(columns={"Animal ID":"animal_id",
                     "Name":"name",
                     "Breed":"breed",
                     "Color":"color",
                     "Found Location":"found_location",
                     "Intake Type":"in_type",
                     "Intake Condition":"in_condition",
                     "Animal Type":"animal_type",
                     "Sex upon Intake":"in_sex",
                     "Age upon Intake":"in_age"}, inplace=True)

In [15]:
#
data[["aui1", "aui2"]] = data["in_age"].str.split(n=1, expand=True)
data["aui1"] = data["aui1"].astype("int")
data.loc[data["aui1"]<0, "aui1"] = 0

In [16]:
conditions = [(data["aui2"]=="day") | (data["aui2"]=="days"),
              (data["aui2"]=="week") | (data["aui2"]=="weeks"),
              (data["aui2"]=="month") | (data["aui2"]=="months"),
              (data["aui2"]=="year") | (data["aui2"]=="years"),]
values = [1/30, 1/(30/7), 1, 12]
data["aui3"] = np.select(conditions, values)

In [17]:
#
data["in_age2"] = data["aui1"] * data["aui3"]
data.drop(["aui1", "aui2", "aui3"], axis=1, inplace=True)

In [18]:
data.sort_values(by="in_date", inplace=True)

<b>RECHECK data</b>

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128748 entries, 39880 to 128739
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   animal_id       128748 non-null  object        
 1   name            128748 non-null  object        
 2   found_location  128748 non-null  object        
 3   in_type         128748 non-null  object        
 4   in_condition    128748 non-null  object        
 5   animal_type     128748 non-null  object        
 6   in_sex          128748 non-null  object        
 7   in_age          128748 non-null  object        
 8   breed           128748 non-null  object        
 9   color           128748 non-null  object        
 10  in_date         128748 non-null  datetime64[ns]
 11  in_time1        128748 non-null  object        
 12  in_time2        128748 non-null  object        
 13  in_age2         128748 non-null  float64       
dtypes: datetime64[ns](1), float64(1)

In [21]:
data["in_type"].unique()

array(['Stray', 'Owner Surrender', 'Public Assist', 'Euthanasia Request',
       'Wildlife', 'Abandoned'], dtype=object)

In [22]:
data["in_condition"].unique()

array(['Normal', 'Nursing', 'Injured', 'Sick', 'Other', 'Aged',
       'Pregnant', 'Feral', 'Medical', 'Behavior', 'Space'], dtype=object)

In [23]:
data["in_sex"].unique()

array(['Intact Male', 'Intact Female', 'Spayed Female', 'Neutered Male',
       'Unknown'], dtype=object)

In [17]:
#data.groupby("animal_id").count().sort_values(by="name").tail(50)

In [18]:
#data[data["animal_id"]=="A700407"].sort_values(by="in_date")

In [19]:
#len(data[data["animal_id"]=="A700407"])

<b>EXPORT data</b>

In [20]:
#
data.to_csv("preprocessed_AnimalCenter-Intakes.csv", index=False)